In [26]:
# imports
import requests
import json
import pandas as pd
import os

In [27]:
bike_stations_df = pd.read_csv('bike_stations.csv', index_col = 0)

In [19]:
bike_stations_df

,Station Name,Available Bikes,Empty Slots,Latitude,Longitude
0,W 100 St & Manhattan Ave,5,29,40.795000,-73.964500
1,7 Ave & Central Park South,2,76,40.766741,-73.979069
2,Bedford Ave & Bergen St,2,15,40.676368,-73.952918
3,28 St & 41 Ave,12,12,40.751047,-73.937970
4,Frederick Douglass Blvd & W 112 St,14,14,40.801694,-73.957145
...,...,...,...,...,...
1974,99 St & Astoria Blvd,0,22,40.761960,-73.870420
1975,E 34 St & Church Ave,6,19,40.651160,-73.945770
1976,Snyder Ave & Schenectady Ave,3,16,40.650000,-73.932960
1977,99 St & 32 Ave,5,14,40.758960,-73.869760


In [ ]:
bike_stations1_df = bike_stations_df.iloc[:500]
bike_stations2_df = bike_stations_df.iloc[500:1000]
bike_stations3_df = bike_stations_df.iloc[1000:1500]
bike_stations4_df = bike_stations_df.iloc[1500:]

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [35]:
def query_foursquare_for_places(lat, lng, radius=1000, limit = 50):
    url = f"https://api.foursquare.com/v3/places/search"
    FOURSQUARE_API_VERSION = '20230820'
    params = {
        'v': FOURSQUARE_API_VERSION,
        'll': f"{lat},{lng}",
        'radius': radius
    }
    
    api_key = os.getenv('foursquare_API')
    
    # Create dictionary for headers
    headers = {"Accept": "application/json"}
    # Add key with our API KEY
    headers['Authorization'] = api_key

    try:
        response = requests.get(url, headers=headers, params=params)
        #print(response)
        #print("Response content:", response.content)  # Print the raw response content
        response_data = response.json()
        print(response.json())
        
        if 'results' in response_data:
            places = response_data['results']
            return places
        else:
            print("No Place of Interest found")
            return []         

    except requests.exceptions.RequestException as e:
        print("Error making API request:", e)
        return []

In [36]:
if bike_stations_df is not None:
    for index, row in bike_stations_df.iterrows():
        print(f"Station Name: {row['Station Name']}")
        print(f"Available Bikes: {row['Available Bikes']}")
        print(f"Empty Slots: {row['Empty Slots']}")
        print(f"Latitude: {row['Latitude']}")
        print(f"Longitude: {row['Longitude']}")
        
        query_foursquare_for_places(row['Latitude'], row['Longitude'])

Station Name: W 100 St & Manhattan Ave
Available Bikes: 5
Empty Slots: 29
Latitude: 40.795
Longitude: -73.9645
{'results': [{'fsq_id': '4fdc9a79e4b0735a6deafc25', 'categories': [{'id': 13035, 'name': 'Coffee Shop', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_', 'suffix': '.png'}}], 'chains': [{'id': 'ab4c54c0-d68a-012e-5619-003048cad9da', 'name': 'Starbucks'}], 'distance': 100, 'geocodes': {'main': {'latitude': 40.794937, 'longitude': -73.965633}, 'roof': {'latitude': 40.794937, 'longitude': -73.965633}}, 'link': '/v3/places/4fdc9a79e4b0735a6deafc25', 'location': {'address': '805 Columbus Ave', 'address_extended': 'Frnt 2', 'census_block': '360610185001000', 'country': 'US', 'cross_street': 'at W 100th St', 'dma': 'New York', 'formatted_address': '805 Columbus Ave (at W 100th St), New York, NY 10025', 'locality': 'New York', 'postcode': '10025', 'region': 'NY'}, 'name': 'Starbucks', 'related_places': {}, 'timezone': 'America/New_York'}, {'fsq_id': '4a95cf

KeyboardInterrupt: 

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [8]:
def foursqare_places_of_interest(lat, lng, limit = 50):
    poi = query_foursquare_for_places(lat, lng, radius=1000)
    
    if poi:
        print("Nearby Places of Interest:")
        for places in poi:
            if 'coffee' in places['categories'][0]['name'].lower() or 'restaurant' in places['categories'][0]['name'].lower() or 'bar' in places['categories'][0]['name'].lower() or 'hotel' in places['categories'][0]['name'].lower():
                print(f"Business Category: {places['categories'][0]['name']}")
                if places['chains']:
                    print(f"Business Chain: {places['chains']}")
                print(f"Business Name: {places['name']}")
                print(f"Business Address: {places['location']['formatted_address']}")
                print("-" * 30)
    else:
        print("No Business found")

In [152]:
if bike_stations_df is not None:
    for index, row in bike_stations_df.iterrows():
        print(f"Station Name: {row['Station Name']}")
        print(f"Available Bikes: {row['Available Bikes']}")
        print(f"Empty Slots: {row['Empty Slots']}")
        print(f"Latitude: {row['Latitude']}")
        print(f"Longitude: {row['Longitude']}")
        
        
        foursqare_places_of_interest(row['Latitude'], row['Longitude'])

Station Name: W 100 St & Manhattan Ave
Available Bikes: 3
Empty Slots: 31
Latitude: 40.795
Longitude: -73.9645
Nearby Places of Interest:
Business Category: Coffee Shop
Business Chain: [{'id': 'ab4c54c0-d68a-012e-5619-003048cad9da', 'name': 'Starbucks'}]
Business Name: Starbucks
Business Address: 805 Columbus Ave (at W 100th St), New York, NY 10025
------------------------------
Business Category: Coffee Shop
Business Name: Birch Coffee
Business Address: 750 Columbus Ave (btwn W 96th & W 97th St), New York, NY 10025
------------------------------
Business Category: Vietnamese Restaurant
Business Name: Saiguette
Business Address: 935 Columbus Ave (at W 106th St), New York, NY 10025
------------------------------
Business Category: Wine Bar
Business Name: Buceo 95
Business Address: 201 W 95th St (btwn Amsterdam Ave & Broadway), New York, NY 10025
------------------------------
Station Name: 7 Ave & Central Park South
Available Bikes: 16
Empty Slots: 62
Latitude: 40.7667405590595
Longitud

KeyboardInterrupt: 

Put your parsed results into a DataFrame

In [29]:
business = []

In [30]:
if bike_stations_df is not None:
    for index, row in bike_stations_df.iterrows():
        results = query_foursquare_for_places(row['Latitude'], row['Longitude'], limit = 50)
        
        for each in results:
            if each['categories']:
                Category = each['categories'][0]['name']
            if each['name']:
                Name = each['name']
            if each['location']:
                Location = each['location']['formatted_address']
            
                      
            if 'coffee' in Category.lower() or 'restaurant' in Category.lower() or 'bar' in Category.lower() or 'hotel' in Category.lower():
                business.append({'Station Name': row['Station Name'], 'Business Category': Category, 'Business Name': Name, 'Address': Location})

In [33]:
foursquare_business_df = pd.DataFrame(business)

In [34]:
foursquare_business_df

,Station Name,Business Category,Business Name,Address
0,W 100 St & Manhattan Ave,Coffee Shop,Starbucks,"805 Columbus Ave (at W 100th St), New York, NY..."
1,W 100 St & Manhattan Ave,Coffee Shop,Birch Coffee,"750 Columbus Ave (btwn W 96th & W 97th St), Ne..."
2,W 100 St & Manhattan Ave,Dive Bar,Bob's Your Uncle,"929 Columbus Ave (btwn W 105th & W 106th St), ..."
3,W 100 St & Manhattan Ave,Vietnamese Restaurant,Saiguette,"935 Columbus Ave (at W 106th St), New York, NY..."
4,W 100 St & Manhattan Ave,Italian Restaurant,Osteria 106,"53 W 106th St (Btwn Manhattan and Columbus), N..."
...,...,...,...,...
7728,32 Ave & 77 St,Bar,Cassidy's Ale House,"7502 31st Ave, East Elmhurst, NY 11370"
7729,32 Ave & 77 St,Bar,The Queensboro Restaurant,"8002 Northern Blvd, Jackson Heights, NY 11372"
7730,32 Ave & 77 St,French Restaurant,Bistro Eloise,"7557 31st Ave, East Elmhurst, NY 11370"
7731,32 Ave & 77 St,Cocktail Bar,Bocaito Cafe & Wine Bar,"8115 Northern Blvd, Jackson Heights, NY 11372"


# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [15]:
def query_yelp_for_places(lat, lng, radius = 1000, limit = 10):
    url = "https://api.yelp.com/v3/businesses/search"
    api_key = os.getenv('yelp_API')
    params = {
        "latitude": lat,
        "longitude": lng,
        "radius": radius
    }
    
    headers = {
        "Authorization": f"Bearer {api_key}"
    }
    
    try:
        response = requests.get(url, params=params, headers=headers)
        #print(response)
        #print(response.content)
        response_data= response.json()
        
        if 'businesses' in response_data:
            places = response_data['businesses']
            return places
        else:
            print("No Place of Interest found")
            return []
    except requests.exceptions.RequestException as e:
        print("Error making API request:", e)
        return []

In [220]:
if bike_stations_df is not None:
    for index, row in bike_stations_df.iterrows():
        print(f"Station Name: {row['Station Name']}")
        print(f"Available Bikes: {row['Available Bikes']}")
        print(f"Empty Slots: {row['Empty Slots']}")
        print(f"Latitude: {row['Latitude']}")
        print(f"Longitude: {row['Longitude']}")
        
        query_yelp_for_places(row['Latitude'], row['Longitude'])

Station Name: W 100 St & Manhattan Ave
Available Bikes: 5
Empty Slots: 29
Latitude: 40.795
Longitude: -73.9645
<Response [429]>
No Place of Interest found
Station Name: 7 Ave & Central Park South
Available Bikes: 2
Empty Slots: 76
Latitude: 40.7667405590595
Longitude: -73.9790689945221
<Response [429]>
No Place of Interest found
Station Name: Bedford Ave & Bergen St
Available Bikes: 2
Empty Slots: 15
Latitude: 40.676368
Longitude: -73.952918
<Response [429]>
No Place of Interest found
Station Name: 28 St & 41 Ave
Available Bikes: 12
Empty Slots: 12
Latitude: 40.751047
Longitude: -73.93797
<Response [429]>
No Place of Interest found
Station Name: Frederick Douglass Blvd & W 112 St
Available Bikes: 14
Empty Slots: 14
Latitude: 40.801694
Longitude: -73.957145
<Response [429]>
No Place of Interest found
Station Name: 3 Ave & E 112 St
Available Bikes: 16
Empty Slots: 5
Latitude: 40.795508
Longitude: -73.941606
<Response [429]>
No Place of Interest found
Station Name: 8 Ave & W 31 St
Availab

KeyboardInterrupt: 

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [155]:
def yelp_places_of_interest(lat, lng, limit = 50):
    poi = query_yelp_for_places(lat, lng, radius = 1000, limit = 50)
    
    if poi:
        print("Nearby Places of Interest:")
        for places in poi:
            print(f"Business Category: {places['categories'][0]['title']}")
            print(f"Business Name: {places['name']}")
            print(f"No of Reviews: {places['review_count']}")
            print(f"Rating: {places['rating']}")
            print(f"Business Address: {places['location']['display_address'][0]}")
            print(f"Business Contact: {places['display_phone']}")
            print("-" * 30)
    else:
        print("No Business found")

In [156]:
if bike_stations_df is not None:
    for index, row in bike_stations_df.iterrows():
        print(f"Station Name: {row['Station Name']}")
        print(f"Available Bikes: {row['Available Bikes']}")
        print(f"Empty Slots: {row['Empty Slots']}")
        print(f"Latitude: {row['Latitude']}")
        print(f"Longitude: {row['Longitude']}")
        
        yelp_places_of_interest(row['Latitude'], row['Longitude'])

Station Name: W 100 St & Manhattan Ave
Available Bikes: 3
Empty Slots: 31
Latitude: 40.795
Longitude: -73.9645
<Response [200]>
Nearby Places of Interest:
Business Category: Comfort Food
Business Name: Jacob's Pickles
No of Reviews: 5402
Rating: 4.0
Business Address: 509 Amsterdam Ave
Business Contact: (212) 470-5566
------------------------------
Business Category: Vietnamese
Business Name: Saiguette
No of Reviews: 729
Rating: 4.5
Business Address: 935 Columbus Ave
Business Contact: (212) 866-6888
------------------------------
Business Category: Sardinian
Business Name: Arco Cafe
No of Reviews: 352
Rating: 4.5
Business Address: 886 Amsterdam Ave
Business Contact: (646) 781-9080
------------------------------
Business Category: Cocktail Bars
Business Name: Nobody Told Me
No of Reviews: 236
Rating: 4.5
Business Address: 951 Amsterdam Ave
Business Contact: 
------------------------------
Business Category: Bars
Business Name: The Calaveras
No of Reviews: 231
Rating: 4.5
Business Address

KeyboardInterrupt: 

Put your parsed results into a DataFrame

In [250]:
restaurants = []

In [253]:
if bike_stations1_df is not None:
    for index, row in bike_stations1_df.iterrows():
        results = query_yelp_for_places(row['Latitude'], row['Longitude'], radius = 1000, limit = 50)
        
        for each in results:
            if each['categories']:
                Category = each['categories'][0]['title']
            if each['name']:
                Name = each['name']
            if each['review_count']:
                Count_of_Reviews = each['review_count']
            if each['rating']:
                Rating = each['rating']
            if each['location']['display_address']:
                Address = each['location']['display_address'][0]
            if each['display_phone']:
                Contact = each['display_phone']
                
            restaurants.append({'Station Name': row['Station Name'], 'Station Latitude': row['Latitude'], 'Station Longitude': row['Longitude'], 'Business Category': Category, 'Business Name': Name, 'Rating': Rating, 'Number of Reviews': Count_of_Reviews, 'Address': Address, 'Contact': Contact})

In [267]:
if bike_stations2_df is not None:
    for index, row in bike_stations2_df.iterrows():
        results = query_yelp_for_places(row['Latitude'], row['Longitude'], radius = 1000, limit = 50)
        
        for each in results:
            if each['categories']:
                Category = each['categories'][0]['title']
            if each['name']:
                Name = each['name']
            if each['review_count']:
                Count_of_Reviews = each['review_count']
            if each['rating']:
                Rating = each['rating']
            if each['location']['display_address']:
                Address = each['location']['display_address'][0]
            if each['display_phone']:
                Contact = each['display_phone']
                
            restaurants.append({'Station Name': row['Station Name'], 'Station Latitude': row['Latitude'], 'Station Longitude': row['Longitude'], 'Business Category': Category, 'Business Name': Name, 'Rating': Rating, 'Number of Reviews': Count_of_Reviews, 'Address': Address, 'Contact': Contact})

In [270]:
yelp_business_df = pd.DataFrame(restaurants)

In [272]:
yelp_business_df.to_csv('yelp_business.csv')

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Yelp API provided me with a lot more data than Four Square API. I was only able to extract three data points from Four Square: The business name, category and location. Other data provided does not provide any value to the current exercise. On the other hand, Yelp offers additional information such as the number of reviews, the rating, type of transactions offered (pick up or delivery), the currency accepted and a phone number to contact the business.

Get the top 10 restaurants according to their rating

In [273]:
Top_10_Restaurants = yelp_business_df.sort_values(by='Rating', ascending = False).head(10)
print(Top_10_Restaurants)

                         Station Name  Station Latitude  Station Longitude  \
23003    St. Ann's Av & Bruckner Blvd         40.803399         -73.919763   
27694                 E 11 St & 1 Ave         40.729538         -73.984267   
11637         Cypress Ave & George St         40.698120         -73.902710   
13385          Melrose Ave & E 150 St         40.816827         -73.917338   
15401        Broadway & Kosciuszko St         40.693290         -73.928520   
21942  Willoughby Ave & Onderdonk Ave         40.709800         -73.917270   
20612        Frost St & Debevoise Ave         40.718820         -73.939480   
20610        Frost St & Debevoise Ave         40.718820         -73.939480   
6559               Carroll St & 5 Ave         40.675162         -73.981483   
7546       Kingston Ave & Herkimer St         40.678907         -73.941428   

        Business Category             Business Name  Rating  \
23003              French                MAiSONETTA     5.0   
27694          